In [2]:
import os
import pandas as pd
import numpy as np
from sim.core.cascade import RepoCascade
from sim.core.inputs import load_inputs, load_k_covid
from sim.ind import ModelPupr, get_intv

In [3]:
iso = 'IND'
in_folder = f'out/B_{iso}'
out_folder = f'out/I_{iso}'

In [4]:
os.makedirs(out_folder, exist_ok=True)

In [5]:
pars_cs = pd.read_csv(f'pars/{iso}/pars_cas.csv')
repo_cs = RepoCascade(pars_cs, 2010)
k_covid = load_k_covid(f'{in_folder}/k_covid.csv')

inp = load_inputs(f'data/pars/{iso}')
model = ModelPupr(inp)

In [6]:
particles = pd.read_csv(f'{in_folder}/Post.csv')
particles = [dict(row) for _, row in particles.iterrows()]

particles = particles

In [7]:
baseline = list()
for i, p in enumerate(particles):
    p['cas'] = repo_cs.sample()
    p['k_covid'] = k_covid
    ys, ms, _ = model.simulate_to_fit(p, t_eval=np.linspace(2015, 2022, 8))
    baseline.append((p, ys, ms.assign(Key = i, Scenario = 'Baseline')))

In [8]:
pr_hi = 0.16
rat = 3

pr_prev_hi = pr_hi * rat / (pr_hi * rat + (1 - pr_hi))

In [9]:
scs = {
    #'Pub90': lambda p: get_intv(p, pdx0_pub=0.9, pdx1_pub=0.9),
    'Dx90': lambda p: get_intv(p, pdx0_pub=0.9, pdx1_pub=0.9, pdx0_eng=0.9, pdx1_eng=0.9),
    'PPM90': lambda p: get_intv(p, pdx0_pub=0.9, pdx1_pub=0.9, pdx0_eng=0.9, pdx1_eng=0.9, ppm=0.9),
    'RedDelay2': lambda p: get_intv(p, rr_csi=2, rr_recsi=2), 
    'ACF2': lambda p: get_intv(p, rd_csi=2, rd_recsi=2),
    'PPM90_RedDelay2': lambda p: get_intv(p, pdx0_pub=0.9, pdx1_pub=0.9, pdx0_eng=0.9, pdx1_eng=0.9, ppm=0.9, rr_csi=2, rr_recsi=2),
    'Dx90_ACF2': lambda p: get_intv(p, pdx0_pub=0.9, pdx1_pub=0.9, pdx0_eng=0.9, pdx1_eng=0.9, rd_csi=2, rd_recsi=2),
    'PPM90_ACF2': lambda p: get_intv(p, pdx0_pub=0.9, pdx1_pub=0.9, pdx0_eng=0.9, pdx1_eng=0.9, ppm=0.9, 
                                     rd_csi=2, rd_recsi=2, r_asym_acf=pr_prev_hi * 0.1),
    'AsymACF': lambda p: get_intv(p, r_asym_acf=pr_prev_hi * 0.1),
}

In [10]:
intvs = list()

for i, (p, ys, _) in enumerate(baseline):
    _, ms0, _ = model.simulate_intv(p, ys)
    intvs.append(ms0.assign(Key = i, Scenario = 'Baseline'))
    
    for k, sc in scs.items():
        intv = sc(p)
        _, ms1, _ = model.simulate_intv(p, ys, intv)
        intvs.append(ms1.assign(Key = i, Scenario = k))

intvs = pd.concat(intvs)

In [11]:
mss0 = [ms for (_, _, ms) in baseline]
mss0 = pd.concat(mss0)
mss0.to_csv(f'{out_folder}/RunBaseline.csv')

In [12]:
intvs.to_csv(f'{out_folder}/RunIntv.csv')

In [13]:
len(intvs)

37800